In [10]:
import torch
from meld_dataset import prepare_dataloaders
import os
from models import MultimodalSentimentModel, MultimodalTrainer
from tqdm import tqdm

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    memory_used = torch.cuda.max_memory_allocated() / 1024**3
    print(f"Initial GPU memory used: {memory_used} GB")

cuda
Initial GPU memory used: 0.5390219688415527 GB


In [12]:
model_dir = "../checkpoints"

train_dir_csv = "../dataset/train/train_sent_emo.csv"
train_dir_video = "../dataset/train/train_splits"
dev_dir_csv = "../dataset/dev/dev_sent_emo.csv"
dev_dir_video = "../dataset\dev\dev_splits_complete"
test_dir_csv = "../dataset/test/test_sent_emo.csv"
test_dir_video = "../dataset/test\output_repeated_splits_test"

batch_size = 8
epochs = 10

In [13]:
train_loader, val_loader, test_loader = prepare_dataloaders(
    train_csv=train_dir_csv,
    train_video_dir=train_dir_video,
    dev_csv=dev_dir_csv,
    dev_video_dir=dev_dir_video,
    test_csv=test_dir_csv,
    test_video_dir=test_dir_video,
    batch_size=batch_size
)

In [14]:
model = MultimodalSentimentModel().to(device)
trainer = MultimodalTrainer(model, train_loader, val_loader)

e:\Projects\Video_Sentiment\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Projects\Video_Sentiment\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Dataset sizes:
Training samples: 9,989
Validation samples: 1,109
Batches per epoch: 1,249


In [15]:
best_val_loss = float('inf')
metrics_data = {
    "train_losses": [],
    "val_losses": [],
    "epochs": []
}

for epoch in tqdm(range(epochs), desc='Epochs'):
        train_loss = trainer.train_epoch()
        val_loss, val_metrics = trainer.evaluate(val_loader)

        # Track metrics 
        metrics_data["train_losses"].append(train_loss['total'])
        metrics_data["val_losses"].append(val_loss['total'])
        metrics_data["epochs"].append(epoch)

        if torch.cuda.is_available():
            memory_used = torch.cuda.max_memory_allocated() / 1024**3
            print(f"Peak GPU memory used: {memory_used:.2f} GB")

        # Save best model
        if val_loss['total'] < best_val_loss:
            best_val_loss = val_loss['total']
            torch.save(model.state_dict(), os.path.join(model_dir, "model.pth"))


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]


TypeError: 'DataLoader' object is not subscriptable

In [ ]:
print(f"Evaluating on test set...")
test_loss, test_metrics = trainer.evaluate(test_loader, phase='test')